In [1]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
import re
from nltk.tokenize import sent_tokenize
#filter(None, re.split('(-|:|am|pm)', '30'))


#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [2]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [4]:
question_class = 'q2'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of comments', len(content)
f.close()

length of comments 478


# Split Reviews based on time

In [20]:
def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def num_there(s):
    return any(i.isdigit() for i in s)

def process_corpus(x1):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_single = []
    for idx, x in enumerate(x1):
        if 'no improvement' in x or 'respondent' in x:
            doc_noimprove.append((x, idx))
        else:
            sents = sent_tokenize(x)
            if len(sents) > 1:
                for sen in sents:
                    doc_single.append((sen, idx))
            else:
                doc_single.append((sents[0], idx))
    return doc_noimprove, doc_single
    #return doc_noimprove, [doc_nn, nn_extracted], doc_other, doc_multi

def process_sentence(x1, day_senswords, time_senswords, general_stop):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_days = []
    doc_time = []
    doc_other = []
    doc_nn = []
    for x in x1:
        nn_list = []
        sen = x[0]
        word_list = nltk.word_tokenize(sen)
        if not set(word_list).isdisjoint(day_senswords):
            doc_days.append((sen, x[1]))
        else:
            clean_word_list = []
            for word in word_list:
                clean_word_list = clean_word_list + filter(None, re.split('(-|:|am|pm)', word)) 
            if 'am' in clean_word_list or 'pm' in clean_word_list or num_there(clean_word_list):
                doc_time.append((sen, x[1]))
            else:
                doc_other.append((sen, x[1]))

    return doc_days, doc_time, doc_other
nn_corpus = []

t_noimporve, t_single = process_corpus(content)


# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]

for punctuation in punctuation_list:
    stop_words.append(punctuation)

day_sense = ['sunday', 'sundays', 'weekend', 'weekends', 'holidays', 'holiday']
time_sense = ['am', 'pm',]
# return day-covered, time-covered, and others data
review_day, review_time, review_other = process_sentence(t_single, day_sense, time_sense, stop_words)


In [34]:
def extract_am(word_list):
    if 'am' in word_list:
        amidxs = [i for i,val in enumerate(word_list) if val=='am']
        time_nn = []
        #print len(amidxs)
        for amidx in amidxs:
            time_slot = []
            idx_start = max(amidx-2, 0)
            for ix in range(idx_start, amidx):
                if word_list[ix].isdigit():
                    time_slot.append(word_list[ix])
            if len(time_slot) == 2:
                time_value = float(time_slot[0])+int(time_slot[1])/60.0
                time_nn.append(time_value)
            if len(time_slot) == 1:
                time_nn.append(float(time_slot[0]))
        if time_nn:
            return min(time_nn)
        else:
            return None

def extract_pm(word_list):
    if 'pm' in word_list:
        amidxs = [i for i,val in enumerate(word_list) if val=='pm']
        time_nn = []
        #print len(amidxs)
        for amidx in amidxs:
            time_slot = []
            idx_start = max(amidx-2, 0)
            for ix in range(idx_start, amidx):
                if word_list[ix].isdigit():
                    time_slot.append(word_list[ix])
            if len(time_slot) == 2:
                time_value = int(time_slot[0])+int(time_slot[1])/60.0
                time_nn.append(float(time_value))
            if len(time_slot) == 1:
                time_nn.append(float(time_slot[0]))
        if time_nn:
            #print word_list
            return max(time_nn)
        else:
            return None

def time_extract(sen):
    word_list = nltk.word_tokenize(sen[0])
    clean_word_list = []
    for word in word_list:
        clean_word_list = clean_word_list + filter(None, re.split('(-|:|am|pm)', word)) 
    return (extract_am(clean_word_list), extract_pm(clean_word_list))

def day_extract(sen): 
    word_list = nltk.word_tokenize(sen[0])
    class_0 = ['sunday', 'sundays']
    class_1 = ['weekend', 'weekends']
    class_2 = ['holidays', 'holiday']
    if not set(word_list).isdisjoint(class_0):
        return class_0[0]
    if not set(word_list).isdisjoint(class_1):
        return class_1[0]
    if not set(word_list).isdisjoint(class_2):
        return class_2[0]


for idx, sing_review in enumerate(review_day):
    review_day[idx] = (sing_review[0], sing_review[1], day_extract(sing_review))

for idx, sing_review in enumerate(review_time):
    review_time[idx] = (sing_review[0], sing_review[1], time_extract(sing_review))

    


In [36]:

num_sen = len(review_day+review_time+t_noimporve)

review_id = [sig[1] for sig in review_day] + [sig[1] for sig in review_time] + [sig[1] for sig in t_noimporve]
review_cat =  [sig[2] for sig in review_day] + [sig[2] for sig in review_time]
review_id = set(review_id)
review_cat = set(review_cat)

print '# of comments', len(content)
print '# of sents', (num_sen+len(review_other))
print '# of clusters', len(review_cat) + 1  # add no improve
print 'sent ratio', float(num_sen)/(num_sen+len(review_other))
print 'comment ratio', len(review_id)/float(len(content))

# of comments 478
# of sents 1060
# of clusters 51
sent ratio 0.625471698113
comment ratio 0.830543933054


# Split Large Cluster